In [2]:
import numpy as np
import pandas as pd
import json

ipl_matches = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRy2DUdUbaKx_Co9F0FSnIlyS-8kp4aKv_I0-qzNeghiZHAI_hw94gKG22XTxNJHMFnFVKsO4xWOdIs/pub?gid=1655759976&single=true&output=csv"
matches = pd.read_csv(ipl_matches)
# matches = matches[~((matches['Team1'] == 'Gujarat Titans') | (matches['Team2'] == 'Gujarat Titans'))]
matches

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,N,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,N,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,N,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar
948,335983,Chandigarh,2008-04-19,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,N,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri


In [2]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
def team_vs_team(team1,team2):
    temp_dict = {}
    df = matches[((matches['Team1'] == team1) & (matches['Team2']  == team2)) | ((matches['Team2']  == team1) & (matches['Team1']  == team2))]
    matches_played = df.shape[0]
    team1_won = df[df['WinningTeam'] == team1].shape[0]
    team2_won = df[df['WinningTeam'] == team2].shape[0]
    draws = matches_played - team1_won - team2_won
    
    temp_dict = {
     'total_matches':matches_played,
      team1 : int(team1_won),
      team2 : int(team2_won),
      'draws' : int(draws)    
    }
    return temp_dict


In [125]:
team_vs_team('Mumbai Indians','Rajasthan Royals')
# team_vs_team('Mumbai Indians','Lucknow Super Giants')

{'total_matches': 26, 'Mumbai Indians': 14, 'Rajasthan Royals': 12, 'draws': 0}


{'total_matches': 26, 'Mumbai Indians': 14, 'Rajasthan Royals': 12, 'draws': 0}

In [43]:
def allRecord(team):
    df = matches[(matches['Team1'] == team) | (matches['Team2'] == team)].copy()
    mp = df.shape[0]
    won = df[df.WinningTeam == team].shape[0]
    nr = df[df.WinningTeam.isnull()].shape[0]
    loss = mp - won - nr
    nt = df[(df.MatchNumber == 'Final') & (df.WinningTeam == team)].shape[0]
    return {'matchesplayed': mp,
            'won': won,
            'loss': loss,
            'noResult': nr,
            'title': nt}


In [ ]:
def teamAPI(team, matches=matches):
    df = matches[(matches['Team1'] == team) | (matches['Team2'] == team)].copy()
    self_record = allRecord(team)
    TEAMS = matches.Team1.unique()
    against = {team2: team_vs_team(team, team2) for team2 in TEAMS if team2 != team}
    data = {team: {'overall': self_record,
                   'against': against}}
    return json.dumps(data, cls=NpEncoder)


In [ ]:
def batsmanRecord(batsman, df):
    if df.empty:
        return np.nan
    out = df[df.player_out == batsman].shape[0]
    df = df[df['batter'] == batsman]
    inngs = df.ID.unique().shape[0]
    runs = df.batsman_run.sum()
    fours = df[(df.batsman_run == 4) & (df.non_boundary == 0)].shape[0]
    sixes = df[(df.batsman_run == 6) & (df.non_boundary == 0)].shape[0]
    if out:
        avg = runs / out
    else:
        avg = np.inf

    nballs = df[~(df.extra_type == 'wides')].shape[0]
    if nballs:
        strike_rate = runs / nballs * 100
    else:
        strike_rate = 0
    gb = df.groupby('ID').sum()
    fifties = gb[(gb.batsman_run >= 50) & (gb.batsman_run < 100)].shape[0]
    hundreds = gb[gb.batsman_run >= 100].shape[0]
    try:
        highest_score = gb.batsman_run.sort_values(ascending=False).head(1).values[0]
        hsindex = gb.batsman_run.sort_values(ascending=False).head(1).index[0]
        if (df[df.ID == hsindex].player_out == batsman).any():
            highest_score = str(highest_score)
        else:
            highest_score = str(highest_score) + '*'
    except:
        highest_score = gb.batsman_run.max()

    not_out = inngs - out
    mom = df[df.Player_of_Match == batsman].drop_duplicates('ID', keep='first').shape[0]
    data = {
        'innings': inngs,
        'runs': int(runs),
        'fours': fours,
        'sixes': sixes,
        'avg': float(avg),
        'strikeRate': float(strike_rate),
        'fifties': fifties,
        'hundreds': hundreds,
        'highestScore': highest_score,
        'notOut': not_out,
        'mom': mom
    }
    
    return data


In [143]:
ipl_ball = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRu6cb6Pj8C9elJc5ubswjVTObommsITlNsFy5X0EiBY7S-lsHEUqx3g_M16r50Ytjc0XQCdGDyzE_Y/pub?output=csv"
balls = pd.read_csv(ipl_ball)

ball_withmatch = balls.merge(matches, on='ID', how='inner').copy()
ball_withmatch['BowlingTeam'] = ball_withmatch.Team1 + ball_withmatch.Team2
ball_withmatch['BowlingTeam'] = ball_withmatch[['BowlingTeam', 'BattingTeam']].apply(lambda x: x.values[0].replace(x.values[1], ''), axis=1)
batter_data = ball_withmatch[np.append(balls.columns.values, ['BowlingTeam', 'Player_of_Match'])]
batter_data

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,BowlingTeam,Player_of_Match
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,HH Pandya
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,HH Pandya
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,HH Pandya
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,HH Pandya
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,HH Pandya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225949,335982,2,14,5,P Kumar,I Sharma,SB Joshi,legbyes,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum
225950,335982,2,14,6,SB Joshi,I Sharma,P Kumar,NaN,1,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum
225951,335982,2,14,7,P Kumar,I Sharma,SB Joshi,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum
225952,335982,2,15,1,SB Joshi,LR Shukla,P Kumar,wides,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum


In [144]:
batsmanRecord('YBK Jaiswal',batter_data)

{'innings': 23,
 'runs': np.int64(547),
 'fours': 62,
 'sixes': 22,
 'avg': 23.782608695652176,
 'strikeRate': 134.72906403940888,
 'fifties': 3,
 'hundreds': 0,
 'highestScore': '68',
 'notOut': 0,
 'mom': 1}

In [145]:
def batsmanVsTeam(batsman, team, df = batter_data):
    df = df[df.BowlingTeam == team].copy()
    return batsmanRecord(batsman, df)

In [146]:
batsmanVsTeam(batsman='YBK Jaiswal',team = 'Mumbai Indians')

{'innings': 4,
 'runs': np.int64(45),
 'fours': 5,
 'sixes': 2,
 'avg': 11.25,
 'strikeRate': 136.36363636363635,
 'fifties': 0,
 'hundreds': 0,
 'highestScore': '32',
 'notOut': 0,
 'mom': 0}

In [ ]:
ipl_ball = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRu6cb6Pj8C9elJc5ubswjVTObommsITlNsFy5X0EiBY7S-lsHEUqx3g_M16r50Ytjc0XQCdGDyzE_Y/pub?output=csv"
data = pd.read_csv(ipl_ball)

In [31]:
data.shape

(225954, 17)

In [ ]:
data.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [12]:
total_batsman = data['batter'].unique()
total_bowler = data['bowler'].unique()

In [21]:
runs = {}
for item in total_batsman:
    runs[item] = int(data[data['batter'] == item]['batsman_run'].sum())

In [28]:
dict(sorted(runs.items(),key = lambda x : x[1],reverse = True))

{'V Kohli': 6634,
 'S Dhawan': 6244,
 'DA Warner': 5883,
 'RG Sharma': 5881,
 'SK Raina': 5536,
 'AB de Villiers': 5181,
 'CH Gayle': 4997,
 'MS Dhoni': 4978,
 'RV Uthappa': 4954,
 'KD Karthik': 4377,
 'G Gambhir': 4217,
 'AT Rayudu': 4190,
 'AM Rahane': 4074,
 'KL Rahul': 3895,
 'SR Watson': 3880,
 'MK Pandey': 3657,
 'SV Samson': 3526,
 'KA Pollard': 3437,
 'F du Plessis': 3403,
 'YK Pathan': 3222,
 'BB McCullum': 2882,
 'RR Pant': 2851,
 'PA Patel': 2848,
 'JC Buttler': 2832,
 'SS Iyer': 2780,
 'Q de Kock': 2767,
 'Yuvraj Singh': 2754,
 'V Sehwag': 2728,
 'SA Yadav': 2644,
 'M Vijay': 2619,
 'RA Jadeja': 2502,
 'SPD Smith': 2495,
 'SE Marsh': 2489,
 'DA Miller': 2455,
 'WP Saha': 2427,
 'JH Kallis': 2427,
 'DR Smith': 2385,
 'MA Agarwal': 2335,
 'SR Tendulkar': 2334,
 'GJ Maxwell': 2320,
 'N Rana': 2181,
 'R Dravid': 2174,
 'KS Williamson': 2105,
 'AJ Finch': 2092,
 'AC Gilchrist': 2069,
 'AD Russell': 2039,
 'JP Duminy': 2029,
 'MEK Hussey': 1977,
 'HH Pandya': 1972,
 'Shubman Gill

In [29]:
wickts = {}
for item in total_bowler:
    wickts[item] = int(data[data['bowler'] == item]['isWicketDelivery'].sum())

In [30]:
dict(sorted(wickts.items(),key = lambda x : x[1],reverse = True))

{'DJ Bravo': 207,
 'SL Malinga': 188,
 'A Mishra': 175,
 'R Ashwin': 174,
 'YS Chahal': 172,
 'SP Narine': 171,
 'B Kumar': 165,
 'PP Chawla': 165,
 'JJ Bumrah': 161,
 'Harbhajan Singh': 161,
 'UT Yadav': 153,
 'RA Jadeja': 141,
 'R Vinay Kumar': 127,
 'Sandeep Sharma': 123,
 'A Nehra': 121,
 'Rashid Khan': 119,
 'Z Khan': 119,
 'Mohammed Shami': 116,
 'AR Patel': 109,
 'K Rabada': 108,
 'CH Morris': 107,
 'SR Watson': 107,
 'HV Patel': 106,
 'DW Steyn': 105,
 'P Kumar': 102,
 'MM Sharma': 100,
 'RP Singh': 100,
 'TA Boult': 99,
 'AD Russell': 99,
 'JD Unadkat': 99,
 'IK Pathan': 99,
 'PP Ojha': 99,
 'JA Morkel': 96,
 'DS Kulkarni': 91,
 'SN Thakur': 88,
 'M Morkel': 88,
 'Imran Tahir': 86,
 'L Balaji': 85,
 'MM Patel': 82,
 'AB Dinda': 82,
 'R Bhatia': 82,
 'KA Pollard': 81,
 'JP Faulkner': 76,
 'MJ McClenaghan': 75,
 'I Sharma': 74,
 'JH Kallis': 74,
 'SK Trivedi': 73,
 'Shakib Al Hasan': 71,
 'M Muralitharan': 67,
 'MG Johnson': 66,
 'KH Pandya': 65,
 'Kuldeep Yadav': 64,
 'KV Sharm

In [32]:
def Records(data=data):
    runs = {}
    wickts = {}
    total_batsman = data['batter'].unique()
    total_bowler = data['bowler'].unique()
    for item in total_batsman:
        runs[item] = int(data[data['batter'] == item]['batsman_run'].sum())

    runs = dict(sorted(runs.items(),key = lambda x : x[1],reverse = True))
    
    for item in total_bowler:
        wickts[item] = int(data[data['bowler'] == item]['isWicketDelivery'].sum())

    wickts = dict(sorted(wickts.items(),key = lambda x : x[1],reverse = True))

    out = {
        'batting-record':runs,
        'bowling-record':wickts
    }
    return out

In [33]:
Records()

{'batting-record': {'V Kohli': 6634,
  'S Dhawan': 6244,
  'DA Warner': 5883,
  'RG Sharma': 5881,
  'SK Raina': 5536,
  'AB de Villiers': 5181,
  'CH Gayle': 4997,
  'MS Dhoni': 4978,
  'RV Uthappa': 4954,
  'KD Karthik': 4377,
  'G Gambhir': 4217,
  'AT Rayudu': 4190,
  'AM Rahane': 4074,
  'KL Rahul': 3895,
  'SR Watson': 3880,
  'MK Pandey': 3657,
  'SV Samson': 3526,
  'KA Pollard': 3437,
  'F du Plessis': 3403,
  'YK Pathan': 3222,
  'BB McCullum': 2882,
  'RR Pant': 2851,
  'PA Patel': 2848,
  'JC Buttler': 2832,
  'SS Iyer': 2780,
  'Q de Kock': 2767,
  'Yuvraj Singh': 2754,
  'V Sehwag': 2728,
  'SA Yadav': 2644,
  'M Vijay': 2619,
  'RA Jadeja': 2502,
  'SPD Smith': 2495,
  'SE Marsh': 2489,
  'DA Miller': 2455,
  'WP Saha': 2427,
  'JH Kallis': 2427,
  'DR Smith': 2385,
  'MA Agarwal': 2335,
  'SR Tendulkar': 2334,
  'GJ Maxwell': 2320,
  'N Rana': 2181,
  'R Dravid': 2174,
  'KS Williamson': 2105,
  'AJ Finch': 2092,
  'AC Gilchrist': 2069,
  'AD Russell': 2039,
  'JP Dumin